RBF网络能够逼近任意非线性的函数。可以处理系统内难以解析的规律性，具有很好的泛化能力，并且具有较快的学习速度。当网络的一个或多个可调参数（权值或阈值）对任何一个输出都有影响时，这样的网络称为全局逼近网络。

由于对于每次输入，网络上的每一个权值都要调整，从而导致全局逼近网络的学习速度很慢，比如BP网络。如果对于输入空间的某个局部区域只有少数几个连接权值影响输出，则该网络称为局部逼近网络，比如RBF网络。接下来重点先介绍RBF网络的原理，然后给出其实现。先看如下图



![rbf structure][id]
[id]: <pic/RBF.bmp> "Optional title"


![rbf description][id]
[id]: <pic/rbf.jpg> "Optional title"

In [2]:
from scipy import *  
from scipy.linalg import norm, pinv  
   
from matplotlib import pyplot as plt  
   
class RBF:  
       
    def __init__(self, indim, numCenters, outdim):  
        self.indim = indim  
        self.outdim = outdim  
        self.numCenters = numCenters  
        self.centers = [random.uniform(-1, 1, indim) for i in xrange(numCenters)]  
        self.beta = 8  
        self.W = random.random((self.numCenters, self.outdim))  
           
    def _basisfunc(self, c, d):  
        assert len(d) == self.indim  
        return exp(-self.beta * norm(c-d)**2)  
       
    def _calcAct(self, X):  
        # calculate activations of RBFs  
        G = zeros((X.shape[0], self.numCenters), float)  
        for ci, c in enumerate(self.centers):  
            for xi, x in enumerate(X):  
                G[xi,ci] = self._basisfunc(c, x)  
        return G  
       
    def train(self, X, Y):  
        """ X: matrix of dimensions n x indim  
            y: column vector of dimension n x 1 """  
           
        # choose random center vectors from training set  
        rnd_idx = random.permutation(X.shape[0])[:self.numCenters]  
        self.centers = [X[i,:] for i in rnd_idx]  
           
        print "center", self.centers  
        # calculate activations of RBFs  
        G = self._calcAct(X)  
        print G  
           
        # calculate output weights (pseudoinverse)  
        self.W = dot(pinv(G), Y)  
           
    def test(self, X):  
        """ X: matrix of dimensions n x indim """  
           
        G = self._calcAct(X)  
        Y = dot(G, self.W)  
        return Y  


if __name__ == '__main__':  
    n = 100  
    x = mgrid[-1:1:complex(0,n)].reshape(n, 1)  
    # set y and add random noise  
    y = sin(3*(x+0.5)**3 - 1)  
    # y += random.normal(0, 0.1, y.shape)  
       
    # rbf regression  
    rbf = RBF(1, 10, 1)  
    rbf.train(x, y)  
    z = rbf.test(x)  
         
    # plot original data  
    plt.figure(figsize=(12, 8))  
    plt.plot(x, y, 'k-')  
       
    # plot learned model  
    plt.plot(x, z, 'r-', linewidth=2)  
       
    # plot rbfs  
    plt.plot(rbf.centers, zeros(rbf.numCenters), 'gs')  
       
    for c in rbf.centers:  
        # RF prediction lines  
        cx = arange(c-0.7, c+0.7, 0.01)  
        cy = [rbf._basisfunc(array([cx_]), array([c])) for cx_ in cx]  
        plt.plot(cx, cy, '-', color='gray', linewidth=0.2)  
       
    plt.xlim(-1.2, 1.2)  
    plt.show()  

center [array([-0.03030303]), array([-0.19191919]), array([ 0.87878788]), array([ 0.39393939]), array([ 0.51515152]), array([-0.93939394]), array([ 0.49494949]), array([-0.83838384]), array([ 0.03030303]), array([ 0.53535354])]
[[  5.40780169e-04   5.38597601e-03   5.44585659e-13   1.77457220e-07
    1.05676022e-08   9.71042777e-01   1.71890731e-08   8.11429096e-01
    2.05062720e-04   6.45452870e-09]
 [  7.37440410e-04   6.97080982e-03   9.96296840e-13   2.77558261e-07
    1.71890731e-08   9.87025018e-01   2.77774606e-08   8.52157700e-01
    2.85167907e-04   1.05676022e-08]
 [  9.99072922e-04   8.96326288e-03   1.81082043e-12   4.31299270e-07
    2.77774606e-08   9.96740351e-01   4.45960818e-08   8.89105812e-01
    3.93984070e-04   1.71890731e-08]
 [  1.34471895e-03   1.14502011e-02   3.26983696e-12   6.65836147e-07
    4.45960818e-08   1.00000000e+00   7.11319879e-08   9.21618123e-01
    5.40780169e-04   2.77774606e-08]
 [  1.79816666e-03   1.45319594e-02   5.86598424e-12   1.0212218